# 📊 Exp C: The Analytical Engine

## Objective
Process logs from Exp A & Exp B to compute paper metrics:
1.  **Trembling Robustness ($R_{noise}$):** (From Exp A) Rate of cooperation collapse when Noise > 0.
2.  **Punishment Rate:** (From Exp B) How often do agents pay to punish?
3.  **Success Rate:** (From Exp B - VD) How often is disaster averted?

In [ ]:
import os
import json
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option('display.max_colwidth', None)

def load_all_results():
    files = [f for f in os.listdir('.') if f.endswith('.json') and 'experiment_results' in f]
    all_data = []
    
    print(f"Found {len(files)} log files.")
    
    for file in files:
        with open(file, 'r', encoding='utf-8') as f:
            data = json.load(f)
            
        for key, val in data.items():
            if 'ERROR' in key: continue
            
            # Extract metadata from key (approximate)
            # Format: Game_Model_Lang_NoiseX
            parts = key.split('_')
            game_type = parts[0]
            
            history = val['history']
            total_rounds = len(history)
            noise_count = 0
            punishment_count = 0
            strategies = []
            
            for r in history.values():
                for agent in r:
                    strategies.append(agent['strategy'])
                    if agent.get('is_noise'):
                        noise_count += 1
                    if agent.get('punished'):
                        punishment_count += 1
            
            all_data.append({
                "File": file,
                "Key": key,
                "Game": game_type,
                "Rounds": total_rounds,
                "NoiseEvents": noise_count,
                "PunishmentActs": punishment_count,
                "CoopRate": sum(1 for s in strategies if 'Coop' in s or 'Contri' in s or 'Volun' in s) / len(strategies) if strategies else 0
            })
            
    return pd.DataFrame(all_data)

df = load_all_results()
if not df.empty:
    display(df[['Key', 'Game', 'Rounds', 'NoiseEvents', 'PunishmentActs', 'CoopRate']])
else:
    print("No data found yet. Run Exp A or B!")